# KBMOD Documentation
This notebook demonstrates the basics of the kbmod image processing and searching python API

Before importing, make sure to run  
`source setup.bash`  
in the root directory, and that you are using the python3 kernel.  
Import everything with:

In [28]:
from kbmod import *
path = "../data/demo/"

# Object Types
There are currently 5 classes exposed to the python interface    
psf   
layered_image  
image_stack  
stack_search  
trajectory

# psf
A psf kernel, for convolution and adding artificial sources to images  
Initializes to a gaussian given the width in pixels

In [2]:
p = psf(1.0)

There are several methods that get information about its properties

In [3]:
p.get_dim() # dimension of kernel width and height
p.get_radius() # distance from center of kernel to edge
p.get_size() # total number of pixels in the kernel
p.get_sum() # total sum of all pixels in the kernel, should be close to 1.0

0.975315511226654

The entire kernel can be printed out (note: prints to the console, not the notebook)

In [42]:
p.print_psf()

# layered_image
Stores the science, mask, and variance image for a single image. The "layered" means it contains all of them together.  
It can be initialized 2 ways:  
A. Generate a new image from scratch

In [5]:
im = layered_image("image1", 100, 100, 20.0, 50.0, 0.0)
# name, width, height, background_noise_sigma, variance, capture_time

B. Load a file

In [29]:
im = layered_image(path+"CORR40535777.fits")

Artificial objects can easily be added into a layered_image

In [7]:
im.add_object(20.0, 35.0, 250.0, p)
# x, y, flux, psf

The image can mask itself by providing a bitmask of flags (note: masked pixels are set to -9999.9 so they can be distinguished later from 0.0 pixles)

In [37]:
flags = int('11001', 2)
# this selects pixels with the first, fourth, and fifth flags (bits read right to left)
im.apply_mask_flags( flags ) 

The image can be convolved with a psf kernel

In [40]:
im.convolve(p)
# note: This function is called interally by stack_search and doesn't need to be
# used directy. It is only exposed because it happens to be a fast 
# implementation of a generally useful function

The image at any point can be saved to a file

In [26]:
im.save_layers(path+"/out/") # file will use original name

Get properties

In [11]:
im.get_width()
im.get_height()
im.get_time()
im.get_ppi() # pixels per image, width*height

8388608

# image_stack
A collection of layered_images. Used to apply operations to a group of images.  

In [17]:
count = 10
imlist = [ layered_image("img"+str(n), 50, 50, 10.0, 5.0, n/count) for n in range(count) ]
stack = image_stack( imlist )
# this creates a stack with 10 50x50 images, and times ranging from 0 to 1

A shortcut is provided to initialize a stack automatically from a list of files

In [33]:
import os
files = os.listdir(path)
files = [path+f for f in files if '.fits' in f]
files.sort()
files

['../data/demo/CORR40535777.fits',
 '../data/demo/CORR40535787.fits',
 '../data/demo/CORR40535797.fits',
 '../data/demo/CORR40535807.fits',
 '../data/demo/CORR40535817.fits']

In [34]:
stack = image_stack(files)

A master mask can be generated and applied to the stack

In [18]:
stack.apply_master_mask( flags, 5) # flags, threshold

Most features of the layered_image can be used on the whole stack

In [38]:
stack.apply_mask_flags(flags)
stack.convolve(p)
stack.get_width()
stack.get_height()
stack.get_ppi()
stack.get_images() # retrieves list of layered_images back from the stack
stack.get_times()

[0.0,
 0.0030539999715983868,
 0.006087000016123056,
 0.00913199968636036,
 0.012175999581813812]

# stack_search
Searches a stack of images for a given psf

In [41]:
search = stack_search( stack, p )

To save psi and images, a directory with "psi" and "phi" folders must be specified.  
To search without saving images, simply skip this step

In [45]:
search.image_save_location("../search/output/")
search.image_save_location("") # <- this will cancel saving

Launch a search

In [47]:
search.gpu(10, 10, 0.0, 1.0, 20.0, 50.0)
# angle_steps, velocity_steps, min_angle, max_angle, min_velocity, max_velocity

Save the results to a files  
note: format is {x, y, xv, yv, likelihood, flux}

In [50]:
search.save_results(path+"results.txt", 0.05) 
# path, fraction of total results to save in file

Trajectories can be retrieved directly from search without writing and reading to file.  
However, this is not recommended for a large number of trajectories, as it is not returned as a numpy array, but as a list of the trajectory objects described below

In [51]:
top_results = search.get_results(0, 100)
# start, count

# trajectory
A simple container with properties representing an object and its path

In [52]:
best = top_results[0]

In [57]:
# these numbers are wild because mask flags and search parameters above were chosen randomly
best.flux 
best.lh
best.x
best.y
best.x_v
best.y_v

2.8951690196990967

tests/test_search.py shows a simple example of how to generate a set of images, add an artificial source, and recover it with search